In [1]:
# Installing the packages
!pip install h5py scikit-learn
!pip install umap-learn
!pip install plotly
!pip install hdbscan

In [2]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121


Looking in indexes: https://download.pytorch.org/whl/cu121
  Using cached https://download.pytorch.org/whl/cu121/torch-2.5.1%2Bcu121-cp311-cp311-linux_x86_64.whl (780.5 MB)
  Using cached https://download.pytorch.org/whl/cu121/torchvision-0.20.1%2Bcu121-cp311-cp311-linux_x86_64.whl (7.3 MB)
  Using cached https://download.pytorch.org/whl/cu121/torchaudio-2.5.1%2Bcu121-cp311-cp311-linux_x86_64.whl (3.4 MB)
  Using cached https://download.pytorch.org/whl/filelock-3.13.1-py3-none-any.whl.metadata (2.8 kB)
  Using cached https://download.pytorch.org/whl/cu121/nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached https://download.pytorch.org/whl/cu121/nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached https://download.pytorch.org/whl/cu121/nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached https://download.pytorch.org/whl/cu121/nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64

In [3]:
# Import packages
import h5py
import numpy as np
import pandas as pd
import gc
import os
import joblib
import hdbscan
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, calinski_harabasz_score
from sklearn.decomposition import PCA
#from sklearn.cluster import DBSCAN
import umap.umap_ as umap
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly.express as px
import plotly.io as pio   
pio.renderers.default = 'iframe'


import umap
#from sklearn.cluster import DBSCAN

from scipy.spatial.distance import cdist
import json

In [4]:
!pip install tensorflow

  Using cached tensorflow-2.19.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.1 kB)
  Using cached absl_py-2.2.2-py3-none-any.whl.metadata (2.6 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-manylinux2010_x86_64.whl.metadata (5.2 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached termcolor-3.1.0-py3-none-any.whl.metadata (6.4 kB)
  Using cached wrapt-1.17.2-cp311-cp311-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.4 kB)
  Using cached grpcio-1.71.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.8 kB)
  Using cached tensorboard-2.19.0-py3-none-any.whl.metadata (1.8 kB)
  Usin

In [2]:
!pip install kaleido

  Using cached kaleido-0.2.1-py2.py3-none-manylinux1_x86_64.whl.metadata (15 kB)
Using cached kaleido-0.2.1-py2.py3-none-manylinux1_x86_64.whl (79.9 MB)


### Performing HDBSCAN Clustering

In [ ]:
import os
import json
import gc
import torch
import numpy as np
import h5py
import hdbscan
import time
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, calinski_harabasz_score
from scipy.spatial.distance import cdist
import umap

# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
# Load the embeddings from the h5 files
def get_embeddings_batch(batch_files, folder_path='Embeddings'):
    all_embeddings = []
    all_file_names_with_indices = []
    for file_name in batch_files:
        file_path = os.path.join(folder_path, file_name)
        with h5py.File(file_path, 'r') as f:
            data_key = list(f.keys())[0]
            embeddings = f[data_key][:]
            embeddings = torch.tensor(embeddings, device=device)
            all_embeddings.append(embeddings)

            parts = file_name.split('_')
            base_name = f"{parts[0]}_{parts[1]}"
            num_embeddings = embeddings.shape[0]
            all_file_names_with_indices.extend([f"{base_name}_{i}" for i in range(num_embeddings)])
        
        #print(f"Loaded {file_name} with {num_embeddings} embeddings")

    combined_embeddings = torch.cat(all_embeddings, dim=0)
    return combined_embeddings, all_file_names_with_indices

def apply_clustering(total_files, folder_path='Embeddings'):
    overall_start = time.time()

    print("\nLoading embeddings...")
    load_start = time.time()
    h5_files = [file for file in os.listdir(folder_path) if file.endswith('.h5')][:total_files]
    embeddings, file_names = get_embeddings_batch(h5_files, folder_path)
    load_end = time.time()
    print(f"Loaded embeddings in {load_end - load_start:.2f} seconds.")
    print(f"Total embeddings: {embeddings.shape[0]}")

    print("\nScaling embeddings...")
    scale_start = time.time()
    embeddings_np = embeddings.cpu().numpy()
# Scaling the embeddings
    global_scaler = StandardScaler()
    scaled_embeddings_np = global_scaler.fit_transform(embeddings_np)
    scale_end = time.time()
    print(f"Scaled embeddings in {scale_end - scale_start:.2f} seconds.")
# Reducing dimensions with UMAP: 758 to 50 dimensions
    print("\nReducing dimensions with UMAP...")
    umap_start = time.time()
    umap_model = umap.UMAP(n_components=50, n_neighbors=50, min_dist=0.05, metric='cosine')
    reduced_embeddings_np = umap_model.fit_transform(scaled_embeddings_np)
    umap_end = time.time()
    print(f"UMAP reduction completed in {umap_end - umap_start:.2f} seconds.")
# Applying HDBSCAN clustering 
    print("\nApplying HDBSCAN clustering...")
    hdbscan_start = time.time()
    clusterer = hdbscan.HDBSCAN(min_samples=30, min_cluster_size=100, cluster_selection_epsilon=0.1)
    labels = clusterer.fit_predict(reduced_embeddings_np)
    hdbscan_end = time.time()
    print(f"HDBSCAN clustering completed in {hdbscan_end - hdbscan_start:.2f} seconds.")
    print(f"HDBSCAN produced {len(set(labels))} clusters (including noise).")
    print(f"  Number of noise points (-1): {np.sum(labels == -1)}")

    # Save pre-evaluation results
    print("\nSaving clustering results before evaluation...")
    np.savez_compressed("full_evaluation_outputs.npz",
                        embeddings=reduced_embeddings_np,
                        labels=labels,
                        unique_ids=np.array(file_names))
    print("Saved to 'pre_evaluation_outputs.npz'")

    # Map files to clusters and compute centroids
    print("\nMapping cluster-to-file assignments and computing centroids...")
    cluster_to_files = {}
    cluster_centroids = {}

    for label in set(labels):
        if label == -1:
            continue
        cluster_indices = np.where(labels == label)[0]
        cluster_embeddings = reduced_embeddings_np[cluster_indices]
        cluster_ids = [file_names[i] for i in cluster_indices]
        cluster_to_files[label] = cluster_ids

        centroid = np.mean(cluster_embeddings, axis=0)
        distances = cdist([centroid], cluster_embeddings, 'euclidean')
        closest_index = np.argmin(distances)
        closest_id = cluster_ids[closest_index]
        cluster_centroids[label] = closest_id

    with open("cluster_to_files_full.json", "w") as f:
        json.dump({int(k): v for k, v in cluster_to_files.items()}, f, indent=4)

    with open("cluster_centroids_full.json", "w") as f:
        json.dump({int(k): v for k, v in cluster_centroids.items()}, f, indent=4)

    print("Cluster mapping and centroids saved.")

    # Checking Number of Resulting Noise Points. 
    noise_ratio = np.sum(labels == -1) / len(labels)
    print(f"  Noise Proportion: {noise_ratio:.2%}")

    gc.collect()
    overall_end = time.time()
    print(f"\nFull clustering process completed in {overall_end - overall_start:.2f} seconds!")

if __name__ == "__main__":
    total_files = 192 
    apply_clustering(total_files=total_files, folder_path='../Embeddings')


2025-05-03 20:46:42.008791: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-03 20:46:42.023739: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746305202.040292     465 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746305202.045313     465 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746305202.059046     465 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

Using device: cuda

Loading embeddings...
Loaded embeddings in 4.81 seconds.
Total embeddings: 2293146

Scaling embeddings...
Scaled embeddings in 22.74 seconds.

Reducing dimensions with UMAP...
UMAP reduction completed in 2845.06 seconds.

Applying HDBSCAN clustering...
HDBSCAN clustering completed in 867.68 seconds.
HDBSCAN produced 101 clusters (including noise).
  Number of noise points (-1): 218660

Saving clustering results before evaluation...
Saved to 'pre_evaluation_outputs.npz'

Mapping cluster-to-file assignments and computing centroids...
Cluster mapping and centroids saved.
  Noise Proportion: 9.54%

Full clustering process completed in 3769.56 seconds!


### Evaluation of the Clustering 

We use silhouette_score and calinski_harabasz_score to evaluate quality of the resulting clusters

In [ ]:
import numpy as np
from sklearn.metrics import silhouette_score, calinski_harabasz_score

def evaluate_clustering(file_path="pre_evaluation_outputs.npz"):
    print(f"Loading clustering results from: {file_path}")
    data = np.load(file_path, allow_pickle=True)

    embeddings = data['embeddings']
    labels = data['labels']
    file_names = data['unique_ids']

    print(f"Loaded {len(labels)} embeddings with {len(set(labels))} unique clusters (including noise).")
    print(f"  Number of noise points (-1): {np.sum(labels == -1)}")
    print(f"  Noise Proportion: {np.sum(labels == -1) / len(labels):.2%}")

    print("\nEvaluating clustering quality...")
    try:
        silhouette = silhouette_score(embeddings, labels)
        print(f"  Silhouette Score: {silhouette:.4f}")
    except Exception as e:
        print(f"  Silhouette Score could not be calculated: {e}")

    try:
        ch_index = calinski_harabasz_score(embeddings, labels)
        print(f"  Calinski-Harabasz Index: {ch_index:.4f}")
    except Exception as e:
        print(f"  Calinski-Harabasz Index could not be calculated: {e}")

if __name__ == "__main__":
    evaluate_clustering("pre_evaluation_outputs.npz")


Loading clustering results from: pre_evaluation_outputs.npz
Loaded 960480 embeddings with 75 unique clusters (including noise).
  Number of noise points (-1): 93475
  Noise Proportion: 9.73%

Evaluating clustering quality...


### Plotly for Visualization

In [11]:
!pip install kaleido

  Using cached kaleido-0.2.1-py2.py3-none-manylinux1_x86_64.whl.metadata (15 kB)
Using cached kaleido-0.2.1-py2.py3-none-manylinux1_x86_64.whl (79.9 MB)


We are plotting resulting clusters using Plotly. In order to be able to display them, initially dimensionality has to be reduced from 50D to 3D

In [ ]:
import kaleido
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import plotly.io as pio
import umap

# Setting renderer for interactive plot
pio.renderers.default = 'notebook'  # or 'browser' if you want it to open in a browser

# Loading saved outputs
outputs = np.load("full_evaluation_outputs.npz") #input your file name here
reduced_embeddings_highd = outputs['embeddings']  # These are still 50D
labels = outputs['labels']

# Perform UMAP reduction to 3D for visualization
print("Reducing embeddings to 3D for visualization...")
umap_3d = umap.UMAP(n_components=3, n_neighbors=40, min_dist=0.5, metric='euclidean')
reduced_embeddings_3d = umap_3d.fit_transform(reduced_embeddings_highd)

# Create a color palette for the clusters
unique_labels = np.unique(labels)
colors = px.colors.qualitative.Plotly  # Default Plotly color scheme
color_map = {label: colors[i % len(colors)] for i, label in enumerate(unique_labels)}

# Build the 3D scatter plot
fig = go.Figure()

for label in unique_labels:
    mask = labels == label
    fig.add_trace(
        go.Scatter3d(
            x=reduced_embeddings_3d[mask, 0],
            y=reduced_embeddings_3d[mask, 1],
            z=reduced_embeddings_3d[mask, 2],
            mode='markers',
            marker=dict(
                size=5,
                color=color_map[label],
                opacity=0.8
            ),
            name=f"Cluster {label}" if label != -1 else "Noise"
        )
    )

# Customize layout
fig.update_layout(
    title="Interactive 3D Plot of HDBSCAN Clustering (UMAP 3D Reduced)",
    scene=dict(
        xaxis_title='UMAP Dimension 1',
        yaxis_title='UMAP Dimension 2',
        zaxis_title='UMAP Dimension 3',
    ),
    legend_title="Clusters",
    width=800,
    height=600,
)

fig.show()
